# famCHESS Study Arm Analyses

Kendra Wyant

## Notes

This script performs analyses of study arm effects for famCHESS study.

## Setup

In [ ]:
options(conflicts.policy = "depends.ok")
suppressMessages(library(tidyverse))
library(Matrix, exclude = c("expand", "pack", "unpack"))
library(lme4)
theme_set(theme_classic()) 

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")

ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

## Data

In [ ]:
data <- read_csv(here::here(path_data, "famchess_data_ana.csv"),
                 show_col_types = FALSE)

## Analyses

From protocol paper:

*Analysis of study arm effects. Linear mixed models will be used to assess effects of study arm on each of our primary and secondary outcomes over time. There will be fixed effects for study arm, time, and their interaction. When applicable, a fixed effect will be added for the baseline values of the outcome variables. A by-subject random intercept and by-subject random slope for time will be included to accommodate the repeated measures on time. Time will be measured quantitatively (in months) and centered at the end of the intervention period (8 months). Baseline scores on the outcomes will be mean centered.*

*Study hypotheses will focus on the test of the parameter estimates for the two study arm effects. We will also test the study arm by time interaction to determine if the study arm effect is consistent or varies in magnitude over the intervention period. If significant study arm by time effects are observed, we will test simple effects of study arm at the timepoints (4, 8, and 12 months).*

*We will consider a number of potential covariates as listed previously. Of this list, those variables that significantly predict our primary outcomes will be included in the final analysis.*

patient primary outcome = `per_heavy_drink_days`  
partner primary outcome = `oq45_scored`

patient secondary outcome = `per_days_abstinent`  
partner secondary outcomes = `relationsatisf_scored` and `abuse_scored`

List of potential covariate variable names: `race_white_only`, `age_c`, `edu_c`, `meetings_yn`, `outpatient_yn_ever`, `er_yn_ever`, `mat_yn`, `inpatient_yn_ever`, `other_psych_treatment_yn`, `other_medications_yn`

### Create contrasts

In [ ]:
col_names <- c("fam_and_achess_vs_C", "achess_vs_fam")
arm_contrasts <- matrix(c(-2/3, 1/3, 1/3, 0, 1/2, -1/2),
                          nrow = 3, ncol = 2,
                          dimnames = list(levels(data$arm), col_names))

data <- data |> 
  mutate(arm_c = as.factor(arm))

contrasts(data$arm_c) <- arm_contrasts
contrasts(data$arm_c)

  fam_and_achess_vs_C achess_vs_fam
0          -0.6666667           0.0
1           0.3333333           0.5
2           0.3333333          -0.5

### Primary Patient Outcome: `per_heavy_drink_days` (Model 1)

*Significant time by arm interaction*

In [ ]:
lmer(per_heavy_drink_days ~ time_c*arm_c + hdd_0 + inpatient_yn_ever + er_yn_ever +
       (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

per heavy drink days 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 27.91 
 23.34 – 32.47 
 <0.001 
 
 
 time c 
 -5.11 
 -7.65 – -2.57 
 <0.001 
 
 
 arm cfam and achess vs C 
 -2.50 
 -10.47 – 5.47 
 0.538 
 
 
 arm cachess vs fam 
 4.81 
 -4.42 – 14.04 
 0.307 
 
 
 hdd 0 
 0.47 
 0.36 – 0.57 
 <0.001 
 
 
 inpatient yn ever 
 -7.22 
 -17.39 – 2.96 
 0.164 
 
 
 er yn ever 
 -8.59 
 -17.47 – 0.28 
 0.058 
 
 
 time c × arm cfam and achess vs C 
 -6.72 
 -12.06 – -1.38 
 0.014 
 
 
 time c × arm cachess vs fam 
 2.32 
 -3.96 – 8.60 
 0.469 
 

 
 N study_id 
 158 
 
 Observations 
 430 
 
 
 Marginal R 2 / Conditional R 2 
 0.288 / 0.761

### Primary Partner Outcome: `oq45_scored` (Model 2-3)

Partner Dyad

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + edu_c + outpatient_yn_ever + 
       other_psych_treatment_yn + other_medications_yn +
       (1 + time_c | study_id), data = subset(data, dyad == 1)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 54.11 
 51.34 – 56.88 
 <0.001 
 
 
 time c 
 -0.86 
 -2.24 – 0.52 
 0.222 
 
 
 arm cfam and achess vs C 
 -2.48 
 -7.07 – 2.10 
 0.287 
 
 
 arm cachess vs fam 
 -1.05 
 -6.26 – 4.17 
 0.693 
 
 
 oq45 0 
 0.71 
 0.61 – 0.81 
 <0.001 
 
 
 edu c 
 -2.34 
 -4.38 – -0.30 
 0.025 
 
 
 outpatient yn ever 
 8.18 
 -0.97 – 17.34 
 0.080 
 
 
 other psych treatment yn 
 3.72 
 -1.43 – 8.87 
 0.156 
 
 
 other medications yn 
 5.35 
 1.18 – 9.53 
 0.012 
 
 
 time c × arm cfam and achess vs C 
 -1.55 
 -4.53 – 1.43 
 0.308 
 
 
 time c × arm cachess vs fam 
 0.75 
 -2.56 – 4.07 
 0.655 
 

 
 N study_id 
 170 
 
 Observations 
 452 
 
 
 Marginal R 2 / Conditional R 2 
 0.508 / 0.767

Patient Dyad

*sig arm effect*

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + mat_yn +
       (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 53.80 
 50.92 – 56.68 
 <0.001 
 
 
 time c 
 -3.35 
 -4.93 – -1.77 
 <0.001 
 
 
 arm cfam and achess vs C 
 -7.11 
 -12.70 – -1.52 
 0.013 
 
 
 arm cachess vs fam 
 0.25 
 -6.11 – 6.61 
 0.939 
 
 
 oq45 0 
 0.69 
 0.58 – 0.79 
 <0.001 
 
 
 mat yn 
 6.86 
 2.05 – 11.68 
 0.005 
 
 
 time c × arm cfam and achess vs C 
 -0.24 
 -3.56 – 3.08 
 0.887 
 
 
 time c × arm cachess vs fam 
 -1.07 
 -4.97 – 2.83 
 0.590 
 

 
 N study_id 
 167 
 
 Observations 
 432 
 
 
 Marginal R 2 / Conditional R 2 
 0.445 / 0.808

### Secondary Patient Outcome: `per_days_abstinent` (Model 3)

In [ ]:
lmer(per_days_abstinent ~ time_c*arm_c + pda_0 + outpatient_yn_ever + 
       meetings_yn + inpatient_yn_ever + (1 + time_c | study_id), 
     data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

per days abstinent 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 56.49 
 51.39 – 61.59 
 <0.001 
 
 
 time c 
 5.93 
 3.46 – 8.39 
 <0.001 
 
 
 arm cfam and achess vs C 
 1.41 
 -7.44 – 10.26 
 0.755 
 
 
 arm cachess vs fam 
 -4.78 
 -15.04 – 5.48 
 0.361 
 
 
 pda 0 
 0.60 
 0.48 – 0.73 
 <0.001 
 
 
 outpatient yn ever 
 4.57 
 -6.10 – 15.25 
 0.400 
 
 
 meetings yn 
 8.22 
 1.28 – 15.16 
 0.020 
 
 
 inpatient yn ever 
 8.45 
 -3.99 – 20.89 
 0.182 
 
 
 time c × arm cfam and achess vs C 
 5.14 
 -0.01 – 10.29 
 0.050 
 
 
 time c × arm cachess vs fam 
 -0.30 
 -6.44 – 5.83 
 0.923 
 

 
 N study_id 
 158 
 
 Observations 
 413 
 
 
 Marginal R 2 / Conditional R 2 
 0.370 / 0.846

### Secondary Partner Outcome 1: `relationsatisf_scored` (Model 4-5)

Partner Dyad

In [ ]:
lmer(relationsatisf_scored ~ time_c*arm_c + relationsatisf_0 + 
       inpatient_yn_ever + (1 + time_c | study_id), 
     data = subset(data, dyad == 1)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

relationsatisf scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 22.07 
 21.43 – 22.72 
 <0.001 
 
 
 time c 
 0.10 
 -0.36 – 0.57 
 0.664 
 
 
 arm cfam and achess vs C 
 -0.53 
 -1.90 – 0.85 
 0.451 
 
 
 arm cachess vs fam 
 -0.21 
 -1.75 – 1.34 
 0.793 
 
 
 relationsatisf 0 
 0.86 
 0.74 – 0.99 
 <0.001 
 
 
 inpatient yn ever 
 -7.23 
 -11.61 – -2.84 
 0.001 
 
 
 time c × arm cfam and achess vs C 
 -0.09 
 -1.10 – 0.92 
 0.858 
 
 
 time c × arm cachess vs fam 
 0.70 
 -0.43 – 1.82 
 0.224 
 

 
 N study_id 
 170 
 
 Observations 
 454 
 
 
 Marginal R 2 / Conditional R 2 
 0.413 / 0.727

Patient Dyad

In [ ]:
lmer(relationsatisf_scored ~ time_c*arm_c + relationsatisf_0 + 
       edu_c + (1 + time_c | study_id), 
     data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

relationsatisf scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 21.41 
 20.76 – 22.06 
 <0.001 
 
 
 time c 
 -0.12 
 -0.53 – 0.29 
 0.573 
 
 
 arm cfam and achess vs C 
 -0.36 
 -1.75 – 1.02 
 0.606 
 
 
 arm cachess vs fam 
 1.31 
 -0.26 – 2.89 
 0.102 
 
 
 relationsatisf 0 
 0.68 
 0.55 – 0.81 
 <0.001 
 
 
 edu c 
 0.61 
 0.03 – 1.20 
 0.040 
 
 
 time c × arm cfam and achess vs C 
 -0.52 
 -1.38 – 0.35 
 0.243 
 
 
 time c × arm cachess vs fam 
 0.61 
 -0.40 – 1.62 
 0.238 
 

 
 N study_id 
 168 
 
 Observations 
 436 
 
 
 Marginal R 2 / Conditional R 2 
 0.318 / 0.723

### Secondary Partner Outcome 2: `abuse_scored` (Model 6)

*Note: large amount of missing data for this outcome*

In [ ]:
lmer(abuse_scored ~ time_c*arm_c + abuse_0 + mat_yn + inpatient_yn_ever + 
       (1 + time_c | study_id), data = subset(data, dyad == 1)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

abuse scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 3.07 
 0.99 – 5.15 
 0.004 
 
 
 time c 
 0.31 
 -0.88 – 1.51 
 0.604 
 
 
 arm cfam and achess vs C 
 0.45 
 -3.83 – 4.72 
 0.837 
 
 
 arm cachess vs fam 
 -2.91 
 -7.99 – 2.17 
 0.259 
 
 
 abuse 0 
 0.38 
 -0.02 – 0.79 
 0.061 
 
 
 mat yn 
 4.98 
 -2.15 – 12.12 
 0.169 
 
 
 inpatient yn ever 
 15.48 
 5.90 – 25.06 
 0.002 
 
 
 time c × arm cfam and achess vs C 
 1.15 
 -1.12 – 3.41 
 0.318 
 
 
 time c × arm cachess vs fam 
 -2.41 
 -5.63 – 0.82 
 0.142 
 

 
 N study_id 
 62 
 
 Observations 
 128 
 
 
 Marginal R 2 / Conditional R 2 
 0.243 / 0.832

In [ ]:
data |> 
  filter(dyad == 1) |> 
  select(abuse_scored, abuse_0) |> 
  naniar::miss_var_summary()

# A tibble: 2 × 3
  variable     n_miss pct_miss
  <chr>         <int>    <num>
1 abuse_scored    310     66.8
2 abuse_0         247     53.2

# A tibble: 2 × 3
  variable     n_miss pct_miss
  <chr>         <int>    <num>
1 abuse_scored    370     79.7
2 abuse_0         291     62.7

## Moderators

From Protocol Paper:  
*Analysis of moderators. Analyses of study arm moderators including patient and partner gender (two variables, each man vs. women), type of relationship at baseline (romantic partner, family, other), partner relationship satisfaction (at baseline) will be conducted in follow-up analyses that add each moderator separately to the linear mixed models described above. Specifically, we will add fixed effects for the moderator and its interactions with study arm contrasts. Study hypotheses about moderation will focus on the test of the parameter estimates for the study arm contrasts by moderator interaction effects.*

Note these moderators differ from figure that list the following:

Patient moderators: `dsm5_criteria`  
Partner moderators: `gender`, `days_app_use`

### Patient Primary Outcome

Patient moderators: `dsm5_criteria`

In [ ]:
lmer(per_heavy_drink_days ~ time_c*arm_c + hdd_0 + inpatient_yn_ever + 
       er_yn_ever + dsm5_criteria + arm_c:dsm5_criteria +
       (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

per heavy drink days 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 30.11 
 5.72 – 54.51 
 0.016 
 
 
 time c 
 -5.11 
 -7.67 – -2.55 
 <0.001 
 
 
 arm cfam and achess vs C 
 -9.65 
 -64.93 – 45.62 
 0.732 
 
 
 arm cachess vs fam 
 1.80 
 -50.97 – 54.56 
 0.947 
 
 
 hdd 0 
 0.47 
 0.36 – 0.57 
 <0.001 
 
 
 inpatient yn ever 
 -7.20 
 -17.82 – 3.41 
 0.183 
 
 
 er yn ever 
 -8.58 
 -17.69 – 0.54 
 0.065 
 
 
 dsm5 criteria 
 -2.10 
 -25.23 – 21.02 
 0.858 
 
 
 time c × arm cfam and achess vs C 
 -6.72 
 -12.11 – -1.32 
 0.015 
 
 
 time c × arm cachess vs fam 
 2.31 
 -4.00 – 8.62 
 0.472 
 
 
 arm cfam and achess vs C × dsm5 criteria 
 7.03 
 -46.39 – 60.45 
 0.796 
 
 
 arm cachess vs fam × dsm5 criteria 
 2.81 
 -47.31 – 52.93 
 0.912 
 

 
 N study_id 
 157 
 
 Observations 
 427 
 
 
 Marginal R 2 / Conditional R 2 
 0.283 / 0.763

### Partner Primary Outcome

Partner moderators: `gender`, `days_app_use`

*Add separately into model.*

Center `days_app_use` and add second dyad gender (`gender_1` and `gender_2`)

In [ ]:
data <- data |> 
  mutate(days_app_use_c = days_app_use - mean(days_app_use, na.rm = TRUE))

gender_1 <- data |> 
  filter(dyad == 1) |> 
  mutate(dyad_id = str_sub(study_id, 1, 3),
         gender_1 = gender) |> 
  select(dyad_id, gender_1) |> 
  unique() 

gender_2 <- data |> 
  filter(dyad == 2) |> 
  mutate(dyad_id = str_sub(study_id, 1, 3),
         gender_2 = gender) |> 
  select(dyad_id, gender_2) |> 
  unique() 

gender <- full_join(gender_1, gender_2, by = "dyad_id")

data <- data |> 
   mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  left_join(gender, by = "dyad_id") 

Partner Gender (`gender_1`) for Partner

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + edu_c + outpatient_yn_ever + 
       other_psych_treatment_yn + other_medications_yn + gender_1 + 
       arm_c:gender_1 + (1 + time_c | study_id), 
     data = subset(data, dyad == 1)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 47.73 
 41.01 – 54.45 
 <0.001 
 
 
 time c 
 -0.90 
 -2.28 – 0.49 
 0.205 
 
 
 arm cfam and achess vs C 
 -3.35 
 -16.95 – 10.25 
 0.629 
 
 
 arm cachess vs fam 
 10.58 
 -4.72 – 25.88 
 0.175 
 
 
 oq45 0 
 0.71 
 0.61 – 0.81 
 <0.001 
 
 
 edu c 
 -2.47 
 -4.49 – -0.44 
 0.017 
 
 
 outpatient yn ever 
 6.56 
 -2.60 – 15.71 
 0.160 
 
 
 other psych treatment yn 
 3.86 
 -1.31 – 9.02 
 0.143 
 
 
 other medications yn 
 6.01 
 1.83 – 10.20 
 0.005 
 
 
 gender 1 
 4.99 
 0.26 – 9.72 
 0.039 
 
 
 time c × arm cfam and achess vs C 
 -1.60 
 -4.59 – 1.38 
 0.292 
 
 
 time c × arm cachess vs fam 
 0.83 
 -2.49 – 4.15 
 0.622 
 
 
 arm cfam and achess vs C × gender 1 
 0.94 
 -8.99 – 10.87 
 0.853 
 
 
 arm cachess vs fam × gender 1 
 -9.44 
 -20.69 – 1.81 
 0.100 
 

 
 N study_id 
 170 
 
 Observations 
 452 
 
 
 Marginal R 2 / Conditional R 2 
 0.519 / 0.769

Partner Gender (`gender_1`) for Patient

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + mat_yn + gender_1 + arm_c:gender_1 +
       (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 54.87 
 47.19 – 62.56 
 <0.001 
 
 
 time c 
 -3.34 
 -4.92 – -1.76 
 <0.001 
 
 
 arm cfam and achess vs C 
 -11.81 
 -28.22 – 4.60 
 0.158 
 
 
 arm cachess vs fam 
 -0.40 
 -19.08 – 18.28 
 0.966 
 
 
 oq45 0 
 0.70 
 0.59 – 0.80 
 <0.001 
 
 
 mat yn 
 6.84 
 2.02 – 11.66 
 0.006 
 
 
 gender 1 
 -0.89 
 -6.49 – 4.71 
 0.756 
 
 
 time c × arm cfam and achess vs C 
 -0.23 
 -3.55 – 3.09 
 0.891 
 
 
 time c × arm cachess vs fam 
 -1.06 
 -4.97 – 2.84 
 0.592 
 
 
 arm cfam and achess vs C × gender 1 
 3.67 
 -8.34 – 15.67 
 0.549 
 
 
 arm cachess vs fam × gender 1 
 0.48 
 -13.11 – 14.07 
 0.944 
 

 
 N study_id 
 167 
 
 Observations 
 432 
 
 
 Marginal R 2 / Conditional R 2 
 0.442 / 0.810

Patient Gender (`gender_2`) for Partner

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + edu_c + outpatient_yn_ever + 
       other_psych_treatment_yn + other_medications_yn + gender_2 + 
       arm_c:gender_2 + (1 + time_c | study_id), 
     data = subset(data, dyad == 1)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 53.56 
 46.00 – 61.12 
 <0.001 
 
 
 time c 
 -0.89 
 -2.28 – 0.50 
 0.209 
 
 
 arm cfam and achess vs C 
 4.77 
 -10.94 – 20.47 
 0.551 
 
 
 arm cachess vs fam 
 -8.98 
 -26.17 – 8.21 
 0.305 
 
 
 oq45 0 
 0.71 
 0.60 – 0.81 
 <0.001 
 
 
 edu c 
 -2.14 
 -4.23 – -0.05 
 0.044 
 
 
 outpatient yn ever 
 7.97 
 -1.37 – 17.31 
 0.094 
 
 
 other psych treatment yn 
 3.76 
 -1.41 – 8.94 
 0.154 
 
 
 other medications yn 
 5.52 
 1.30 – 9.73 
 0.010 
 
 
 gender 2 
 0.30 
 -4.08 – 4.69 
 0.892 
 
 
 time c × arm cfam and achess vs C 
 -1.51 
 -4.53 – 1.51 
 0.325 
 
 
 time c × arm cachess vs fam 
 0.78 
 -2.55 – 4.10 
 0.647 
 
 
 arm cfam and achess vs C × gender 2 
 -4.46 
 -13.85 – 4.92 
 0.351 
 
 
 arm cachess vs fam × gender 2 
 4.99 
 -5.39 – 15.38 
 0.345 
 

 
 N study_id 
 169 
 
 Observations 
 449 
 
 
 Marginal R 2 / Conditional R 2 
 0.508 / 0.767

Patient Gender (`gender_2`) for Patient

*arm by gender interaction*

In [ ]:
lmer(oq45_scored ~ time_c*arm_c + oq45_0 + mat_yn + gender_2 + arm_c:gender_2 +
       (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
  sjPlot::tab_model(show.re.var = F, show.icc = F)

oq 45 scored 
 
 
 Predictors 
 Estimates 
 CI 
 p 
 
 
 (Intercept) 
 46.28 
 37.91 – 54.66 
 <0.001 
 
 
 time c 
 -3.34 
 -4.92 – -1.76 
 <0.001 
 
 
 arm cfam and achess vs C 
 11.20 
 -6.39 – 28.79 
 0.212 
 
 
 arm cachess vs fam 
 -13.95 
 -34.02 – 6.12 
 0.173 
 
 
 oq45 0 
 0.69 
 0.59 – 0.80 
 <0.001 
 
 
 mat yn 
 6.80 
 2.05 – 11.55 
 0.005 
 
 
 gender 2 
 4.76 
 -0.18 – 9.71 
 0.059 
 
 
 time c × arm cfam and achess vs C 
 -0.23 
 -3.55 – 3.09 
 0.893 
 
 
 time c × arm cachess vs fam 
 -1.08 
 -4.98 – 2.82 
 0.587 
 
 
 arm cfam and achess vs C × gender 2 
 -11.47 
 -22.03 – -0.91 
 0.033 
 
 
 arm cachess vs fam × gender 2 
 9.07 
 -3.08 – 21.22 
 0.143 
 

 
 N study_id 
 167 
 
 Observations 
 432 
 
 
 Marginal R 2 / Conditional R 2 
 0.462 / 0.810

`days_app_use` for Partner  
*Too much missing data when app use added?*

In [ ]:
# lmer(oq45_scored ~ time_c*arm_c + oq45_0 + edu_c + outpatient_yn_ever + 
#        other_psych_treatment_yn + other_medications_yn + days_app_use_c +
#        arm_c:days_app_use_c + (1 + time_c | study_id), 
#      data = subset(data, dyad == 1)) |> 
#   sjPlot::tab_model(show.re.var = F, show.icc = F)

data |> 
  filter(dyad == 1) |> 
  select(days_app_use_c) |> 
  naniar::miss_var_summary()

# A tibble: 1 × 3
  variable       n_miss pct_miss
  <chr>           <int>    <num>
1 days_app_use_c    304     65.5

`days_app_use` for Patient  
*Rank deficient error*

In [ ]:
# lmer(oq45_scored ~ time_c*arm_c + oq45_0 + mat_yn + 
#        arm_c:days_app_use_c + days_app_use_c +
#        (1 + time_c | study_id), data = subset(data, dyad == 2)) |> 
#   sjPlot::tab_model(show.re.var = F, show.icc = F)

data |> 
  filter(dyad == 2) |> 
  select(days_app_use_c) |> 
  naniar::miss_var_summary()

# A tibble: 1 × 3
  variable       n_miss pct_miss
  <chr>           <int>    <num>
1 days_app_use_c    151     32.5